In [1]:
# Convolutional Network for Image Classification
# Michael Timbes

In [2]:
import tensorflow as tf
import numpy as np

#Visualization
import matplotlib.pyplot as plt

#File import
from os import listdir
from os import path as opath
from PIL import Image as PImage
import pickle

#Path of train images
train_path = ('NotFace')

#Path of test images
test_path = ('test_set')

IMAG_X = 100

# Keys
keyA = 'face' #True Class
keyB = 'notface' #False Class

In [3]:
#Import data
def ImportImages(path):
    """
    def ImportImages(path):
    USES VARIABLES: IMAG_X, IMAG_Y, expects these to be set already.
    If prefered, edit code to pass them as arguments. Images must be in 
    RGB space.
    ________________________________________________________________
    Function Outline:
    1. Loads list of images from the path variable (type string).
    2. Iterates through directory loads image into I/O file stream.
    3. Converts file Numpy array.
    4. Reads labels and converts to binary class.
    5. Returns Numpy array objects for images and labels.
    """

    imagesList = listdir(path)
    
    loadedImages = []
    
    loadedLabels = []
  
    new_size = IMAG_X, IMAG_X
    
    for image in imagesList:
        if not(".DS_Store" in path +'/'+ image): #Issue in Mac File System
            img = PImage.open(path +'/'+ image)
            label = opath.splitext(image)[:]
        # Pull file name from current image- use it as a label
        
            loadedLabels.append(label[0])
            img.load()
        # Resize step- ensures that all images follow.
            #img.thumbnail(new_size, PImage.ANTIALIAS )
            #img.convert('1')
            #img.resize(new_size)
            loadedImages.append(np.asarray( img, dtype="int32" ))
        
    # Convert to Binary Classification.
    for i in range(0,len(loadedLabels)):
        if (keyA in loadedLabels[i] and not(keyB in loadedLabels[i]) ):
            loadedLabels[i] = [1, 0]
        else:
            loadedLabels[i] = [0, 1]
    
    return np.asarray(loadedImages), np.asarray(loadedLabels)

In [4]:
def weight_variable(shape):
    """
    Helper function to help initialize weight variables.
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


In [5]:
def bias_variable(shape):
    """
    Helper function to initialize bias variable.
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [6]:
def conv2d(x, W):
    """
    Helper function that returns a 2d convolutional layer
    """
    x = x.astype(np.float32, copy=False)
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


In [7]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
def shape_up(data, size):
    """
    shape_up_X(train_X, IMAG_X):
    Expects a NUM_IN * NUM_IN sized picture. Changes
    the shape to be (N,NUM_IN**2).
    ____________________________________
    """
    #num_exs = len(data[:,0,0])
    #for i in range(num_exs):
      #  temp.append(tf.reshape(data[i,:,:], [-1,size,size,1]))
    return np.reshape(data, [-1,size,size,1])

In [9]:
# Data Import
x_tr, y_tr = ImportImages(train_path)

# Data Convert
x_tr = shape_up(x_tr,IMAG_X)
x_tr.astype(float);
print(x_tr.shape)

(123, 100, 100, 1)


In [10]:
# Layer 1

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Convolution of Layer 1
h_conv1 = tf.nn.relu(conv2d(x_tr, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [11]:
# Connected Layer
W_fc1 = weight_variable([50 * 50 * 32, 64])
b_fc1 = bias_variable([64])

h_pool2_flat = tf.reshape(h_pool1, [-1, 50* 50* 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [12]:
# Output Layer
W_fc2 = weight_variable([64, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

In [13]:
# Set Up Training
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))